In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from dash import Dash, html, dash_table, dcc, Output, Input
import dash_bootstrap_components as dbc


# Importing and cleaning data

In [2]:
pp_db_raw = pd.read_csv("global_power_plant_database.csv", encoding='utf-8', dtype={'other_fuel3': str})
cont = pd.read_csv("list-of-countries-by-continent-2024.csv", encoding='utf-8')

Check columns and merge on country_long and country column

In [ ]:

print("Before merge", pp_db_raw.shape)
print("Power Plant DB columns: ", pp_db_raw.columns)
print("Country & Continent CSV file columns: ", cont.columns)
pp_db = pp_db_raw.merge(cont, how='left', left_on='country_long', right_on='country', suffixes=('_short', ''))
pp_db.shape # 3 columns added and same number of rows due to left join


In [ ]:
pp_db.describe()

Check records missing country field to see if merge was successful

In [ ]:

print(pp_db[['country_short', 'country_long', 'country', 'continent']].isna().sum())
# country column in merged table has 73 missing values, we check which records these correspond to in raw table
print(pp_db_raw[pp_db['country'].isna()]['country_long'].unique())
# Note that 2 power plants are located in Antarctica which we add to Oceania (New Zealand)
print("Antarctica Power Plants: ", pp_db.loc[pp_db['country_long']=='Antarctica', :'primary_fuel'])


Comparing output to list-of-countries-by-continent-2024.csv , we make the following changes manually:

Swaziland --> Eswatini, Brunei Darussalam --> Brunei

Congo --> Republic of the Congo, Cote DIvoire --> Ivory Coast

Democratic Republic of the Congo --> DR Congo, Macedonia --> North Macedonia

Syrian Arab Republic --> Syria

We also  make these changes to the raw data in the continent CSV file:

Add Kosovo, United States --> United States of America

Antarctica --> New Zealand


In [ ]:
replacement_dict={
    'Swaziland': 'Eswatini',
    'Brunei Darussalam': 'Brunei',
    'Congo': 'Republic of the Congo',
    'Cote DIvoire': 'Ivory Coast',
    'Democratic Republic of the Congo': 'DR Congo',
    'Macedonia': 'North Macedonia',
    'Syrian Arab Republic': 'Syria',
    'Antarctica': 'New Zealand'
}
pp_db_raw['country_long'] = pp_db_raw['country_long'].replace(replacement_dict)

pp_db = pp_db_raw.merge(cont, how='left', left_on='country_long', right_on='country', suffixes=('_short', ''))

print(pp_db[['country_short', 'country_long', 'country', 'continent']].isna().sum())

# Creating new features for analysis

Creating new columns for renewable vs non-renewable energy and 'Other' primary fuel_type by continent and world categories for data visualisation. Aim is that any primary fuel type that represents less than 1% of the output for a continent will be added to an 'Other' category.

In [ ]:
print(pp_db['primary_fuel'].unique())
renewable_energies = ['Hydro', 'Solar', 'Wind', 'Biomass', 'Wave and Tidal', 'Geothermal']
non_renewable_energies = ['Nuclear', 'Coal', 'Gas', 'Oil', 'Petcoke',]
other = ['Other', 'Waste', 'Cogeneration', 'Storage']

conditions = [
    pp_db['primary_fuel'].isin(renewable_energies),
    pp_db['primary_fuel'].isin(non_renewable_energies),
    pp_db['primary_fuel'].isin(other)
]

# Define the categories
choices = ['Renewable', 'Non-Renewable', 'Other']

# Create new column
pp_db['renew_energy'] = np.select(conditions, choices, default='Unknown')

In [8]:
# Creating new column called text to use as data that shows ehn we hover over points in the world map figure
pp_db['text'] = pp_db['name'] + ', ' + pp_db['country']

# Creating WebApp with Dash and Plotly

### Defining Color Maps

In [9]:
# Defining colour maps to be reused across plots
fuel_color_map = {
    # Renewable Energy Sources
    'Hydro': '#2ECC71',        # Light Green
    'Solar': '#F1C40F',        # Yellow
    'Wind': '#3498DB',         # Light Blue
    'Biomass': '#27AE60',      # Green
    'Wave and Tidal': '#D35400',  # Teal
    'Geothermal': '#9B59B6',   # Purple

    # Non-Renewable Energy Sources
    'Nuclear': '#34495E',      # Dark Blue-Grey
    'Coal': '#E74C3C',         # Red
    'Gas': '#7F8C8D',          # Dark Orange
    'Oil': '#1ABC9C',          # Darker Orange
    'Petcoke': '#95A5A6',      # Grey

    # Other Categories
    'Other': '#BDC3C7',        # Light Grey
    'Waste': '#E67E22',        # Dark Grey
    'Cogeneration': '#16A085', # Dark Teal
    'Storage': '#2980B9',      # Blue

    # Category Labels (Overall)
    'Renewable': '#2ECC71',    # Light Green
    'Non-Renewable': '#E74C3C', # Red
}


In [10]:

custom_colors = [
    "#FF5733",  # Red-Orange
    "#33FF57",  # Green
    "#3357FF",  # Blue
    "#F39C12",  # Orange
    "#9B59B6",  # Purple
    "#E74C3C",  # Red
    "#1ABC9C",  # Teal
    "#8E44AD",  # Dark Purple
    "#3498DB",  # Light Blue
    "#2ECC71",  # Light Green
    "#E67E22",  # Dark Orange
    "#ECF0F1",  # Light Grey
    "#95A5A6",  # Grey
    "#34495E",  # Dark Blue-Grey
    "#16A085"   # Dark Teal
]

### Defining unique plotting functions

In [11]:
# Function for making the title card
def drawTitle():
    return html.Div([
        dbc.Card(
            dbc.CardBody([
                html.Div([
                    html.H1("Dashboard for EDA of Global Power Plants Database"),
                        ], style={'textAlign': 'center',  'color': 'white'}) 
            ])
        ),
    ])

In [12]:
# For making the dropdown option
def drawDropdown():
    return html.Div([
        dbc.Card(
            dbc.CardBody([
                html.Div([
                    html.H3("Select Continent:"),
                ], style={'textAlign': 'center',  'color': 'white'}),
                dcc.Dropdown(id='cont_dropdown',
                        options = [{'label': continent, 'value': continent} for continent in pp_db['continent'].unique()] + [{'label': 'World', 'value': 'All'}],
                        value='Europe'), 
            ])
        ),
    ])

In [13]:
# For making the table
def drawTable():
    return html.Div([
        dbc.Card(
            dbc.CardBody([
                html.H5("Power Plants Table Sorted by Capacity", style={'textAlign': 'center', 'color': 'white'}),
                html.Br(),
                dash_table.DataTable(
                    id='power-plants-table',
                    columns=[
                        {'name': 'Country', 'id': 'country_long'},
                        {'name': 'Name', 'id': 'name'},
                        {'name': 'Capacity (MW)', 'id': 'capacity_mw'},
                        {'name': 'Primary Fuel', 'id': 'primary_fuel'}
                    ],
                    data=[],
                    page_size=13,
                    style_header={
                        'backgroundColor': '#1f1f1f',  # Dark background for header
                        'color': 'white',              # White text for header
                        'border': '1px solid #444'     # Border color for header
                    },
                    style_cell={
                        'backgroundColor': '#333333',  # Dark background for cells
                        'color': 'white',              # White text for cells
                        'textAlign': 'left',           # Align text to the left
                        'border': '1px solid #444'     # Border color for cells
                    },
                    style_data_conditional=[
                        {
                            'if': {'row_index': 'odd'},  # Alternate row styling for better readability
                            'backgroundColor': '#2d2d2d'  # Slightly lighter dark background for odd rows
                        }
                    ],
                    fill_width=True
                )
            ])
        ),
    ])

In [14]:
# For making PieChart
def drawPieChart():
    return  html.Div([
        dbc.Card(
            dbc.CardBody([html.H5('Proportion of Total Power Plant Capacity', style={'textAlign': 'center', 'color': 'white'}),
                          dcc.RadioItems(id='pie_radioitems',
                        options=[{'label':' Primary Fuel', 'value':'primary_fuel'},
                                {'label':' Renewables', 'value':'renew_energy'}],
                        value='primary_fuel',
                        inline=True),
                        dcc.Graph(id='pie_chart_cap'),
            ])
        ),  
    ])

In [15]:
# For making the geo map
map_columns=['country_long', 'name', 'capacity_mw', 'primary_fuel', 'longitude', 'latitude', 'text']
map_df = pp_db.sort_values(by='capacity_mw', ascending=False).head(500)[map_columns]
map_fig = px.scatter_geo(
    data_frame=map_df,
    lat='latitude',
    lon='longitude',
    color='primary_fuel',
    hover_name='text',
    color_discrete_sequence=custom_colors,
    size='capacity_mw',
    opacity=0.7,
    projection='natural earth',
)

# Update marker borders for better visibility
map_fig.update_traces(
    marker=dict(
        line=dict(
            color='black',  
            width=0.1  
        )
    )
)

# Changing map appearance to darkmode
map_fig.update_layout(
    template='plotly_dark',
    plot_bgcolor= 'rgba(0, 0, 0, 0)',
    paper_bgcolor= 'rgba(0, 0, 0, 0)',
    font=dict(color='white'),  
    geo=dict(
        showland=True, landcolor='#333333',  
        showocean=True, oceancolor='#444444', 
        lakecolor='#444444',  
    ),
    title_font=dict(color='white'), 
)

# Draw it in the card

def drawMap():
    return  html.Div([
        dbc.Card(
            dbc.CardBody([html.H5("Distribution of 500 Largest Power Plants by Capacity", style={'textAlign': 'center',  'color': 'white'}),
                         dcc.Graph(figure=map_fig)
            ])
        ),  
    ])

In [16]:
# Make histogram

def drawHist():
    return  html.Div([
        dbc.Card(
            dbc.CardBody([
                        dcc.Graph(id='capacity_hist')
            ])
        ),  
    ])


### Defining the web application

In [17]:
app = Dash(external_stylesheets=[dbc.themes.SLATE])

### Defining app callback functions

In [18]:

# Callback function for histogram
@app.callback(Output('capacity_hist', 'figure'),
              Input('cont_dropdown', 'value'))
def sync_input_hist(cont_selection):
    if cont_selection!='All':
        continent_filtered = pp_db.loc[pp_db['continent'] == cont_selection].copy()
    else:
        continent_filtered = pp_db
    continent_filtered['log_capacity_mw'] = np.log10(continent_filtered.loc[:,'capacity_mw'])
    fig = px.histogram(continent_filtered,  
                       x='log_capacity_mw',
                       hover_name='text',
                       color='primary_fuel',
                       log_y=True,
                       nbins=100)
    
    tick_values = np.log10(np.array([1, 10, 100, 500, 1000, 5000, 10000]))
    tick_texts = ['1', '10', '100', '500', '1K', '5K', '10K']

    fig.update_layout(
        template='plotly_dark',
        plot_bgcolor= 'rgba(0, 0, 0, 0)',
        paper_bgcolor= 'rgba(0, 0, 0, 0)',   # Background color for the plot area
        font=dict(color='white'),  # Font color for axes and labels
        title_font=dict(color='white'),  # Title font color
        xaxis=dict(
            title='Capacity (MW)', showgrid=True,
            gridcolor='rgba(255, 255, 255, 0.1)', color='white',
            tickmode='array', tickvals=tick_values, ticktext=tick_texts,
            range=[0, continent_filtered['log_capacity_mw'].max()+1]  # This sets the range from 10^0 to 10^5
                            ),
        yaxis=dict(
            title='Count', showgrid=True, gridcolor='rgba(255, 255, 255, 0.1)', color='white'
        ),
        bargap=0.1
    )

    fig.update_layout(barmode='overlay')
    return fig

In [19]:
# Callback function for Pie Chart
@app.callback(Output('pie_chart_cap', 'figure'),
              Input('cont_dropdown', 'value'),
              Input('pie_radioitems', 'value'))
def sync_input_pie_chart_cap(cont_selection, fuel_or_renew):
    if cont_selection!='All':
        continent_filtered = pp_db.loc[pp_db['continent'] == cont_selection].copy()
    else:
        continent_filtered = pp_db
    grouped_by_fuel = continent_filtered.groupby(fuel_or_renew)['capacity_mw'].sum().reset_index()
    fig = px.pie(grouped_by_fuel, 
                 values='capacity_mw',
                 names=fuel_or_renew,
                 color=fuel_or_renew,
                 color_discrete_map=fuel_color_map)
    fig.update_layout(
        template='plotly_dark',
        plot_bgcolor= 'rgba(0, 0, 0, 0)',
        paper_bgcolor= 'rgba(0, 0, 0, 0)',  # Background color for the entire plot
        font=dict(color='white'),  # Font color for pie chart labels
        title_font=dict(color='white'),  # Title font color
    )

    return fig

In [20]:
# Callback function for Table

@app.callback(
    Output('power-plants-table', 'data'),
    [Input('cont_dropdown', 'value')]
)
def update_table(cont_selection):
    if cont_selection != 'All':
        filtered_data = pp_db[pp_db['continent'] == cont_selection]
    else:
        filtered_data = pp_db
    
    sorted_data = filtered_data[['country_long', 'name', 'capacity_mw', 'primary_fuel']].sort_values('capacity_mw', ascending=False)

    return sorted_data.to_dict('records')

### App layout and loader

In [ ]:

 # Actual app layout
app.layout = html.Div([
    dbc.Card(
        dbc.CardBody([
            dbc.Row([
                dbc.Col(drawTitle(), width=12),
            ], align='center'),
            dbc.Row([
                dbc.Col(drawDropdown(), width=4),
            ], justify='center', align='center'),
            dbc.Row([
                dbc.Col([
                    drawHist() 
                ], width=6),
                dbc.Col([
                    drawMap()
                ], width=6),
            ], align='center'), 
            dbc.Row([
                dbc.Col([
                    drawPieChart()
                ], width=6),
                dbc.Col([
                    drawTable()
                ], width=6),
            ], align='center'),      
        ]), color = 'dark'
    )
], style={'height': '100vh', 'overflow': 'auto'})

# Run the dashboard
app.run(jupyter_mode="external")
#app.run_server(debug=True, use_reloader=False)  # Turn off reloader if inside Jupyter
